In [2]:
import csv
import datetime
import pickle
import sys
import timeit

import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
INPUT_IN_FILE = 'input/danWhyStopFlaggingIn.txt'
INPUT_OUT_FILE = 'input/danWhyStopFlaggingOut.txt'
INPUT_RECORDS_FILE = 'input/danWhyStopFlaggingRecords.txt'
INPUT_DEMOGRAPHICS_FILE = 'input/danWhyStopFlaggingDemographics.txt'
PREPROCESS_OUTPUT_FILE = 'preprocess_start.txt'
TEST_SIZE = 0.2

In [4]:
script_start_time = datetime.datetime.now()
print('{} started at {}'.format(sys.argv[0], script_start_time))

c:\users\space\.virtualenvs\machine-learning-n1lcc53y\lib\site-packages\ipykernel_launcher.py started at 2018-10-21 16:58:13.443354


In [5]:
# Read data
print('Reading data...', end='')
start_time = timeit.default_timer()
in_df = pd.read_csv(INPUT_IN_FILE, parse_dates=['DateStartedFlagging'])
out_df = pd.read_csv(INPUT_OUT_FILE, parse_dates=['DateStoppedFlagging'])
FLAGGING_START_DATE, FLAGGING_END_DATE = in_df.DateStartedFlagging.min(), out_df.DateStoppedFlagging.max()
demographics_df = pd.read_csv(INPUT_DEMOGRAPHICS_FILE, index_col=['PatID'])
demographics_df.Sex.replace(['F', 'M'], [0, 1], inplace=True)
records_df = pd.read_csv(INPUT_RECORDS_FILE, index_col=['EntryDate'], parse_dates=['EntryDate'], encoding = "ISO-8859-1")
records_df = records_df.loc[records_df.index > '2009-03-30', :]
RECORDS_START_DATE, RECORDS_END_DATE = records_df.index.min(), records_df.index.max()
print(' done in {:.2f}s'.format(timeit.default_timer() - start_time), flush=True)

Reading data... done in 41.62s


In [7]:
records_df.reset_index(inplace=True)

In [9]:
without_dupe_groups_df = records_df.drop_duplicates(subset=['EntryDate', 'PatID', 'ReadCode'])

In [11]:
without_dupe_groups_df.Rubric = without_dupe_groups_df.Rubric.str[:57]

c:\users\space\.virtualenvs\machine-learning-n1lcc53y\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [12]:
without_dupe_readcodes_df = without_dupe_groups_df.drop_duplicates(subset=['EntryDate', 'PatID', 'Rubric'])

In [14]:
without_dupe_readcodes_df.set_index('EntryDate', inplace=True)

In [16]:
without_dupe_readcodes_df.to_csv('input/danWhyStopFlaggingRecordsWithoutDuplicateGroupsOrCodes.txt')